In [1]:
import torch 
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

from src.equation.burgers import Burgers1D
from src.geometry import Rectangle1D

In [2]:
xmax = 1
xmin = -1

domain = Rectangle1D(xmin, xmax)

In [ ]:
def bc_loss(predict, label):